In [9]:
from collections import deque

class Environment:
    def __init__(self, grid):
        self.grid = grid
        self.rows = len(grid)
        self.cols = len(grid[0])
        self.start = self.find_position('S')
        self.key = self.find_position('K')

    def find_position(self, symbol):
        for i in range(self.rows):
            for j in range(self.cols):
                if self.grid[i][j] == symbol:
                    return (i, j)
        return None

    def is_valid(self, x, y):
        if 0 <= x < self.rows and 0 <= y < self.cols and self.grid[x][y] != '#':
            return True
        return False

class GoalBasedAgent:
    def __init__(self, environment):
        self.environment = environment

    def bfs(self):
        directions = [(-1, 0), (1, 0), (0, -1), (0, 1)]  # Up, Down, Left, Right
        queue = deque([(self.environment.start, [self.environment.start])])
        visited = set()

        while queue:
            (x, y), path = queue.popleft()

            if (x, y) == self.environment.key:
                return path

            if (x, y) in visited:
                continue

            visited.add((x, y))

            for dx, dy in directions:
                nx, ny = x + dx, y + dy

                if self.environment.is_valid(nx, ny) and (nx, ny) not in visited:
                    queue.append(((nx, ny), path + [(nx, ny)]))

        return None

# Example Grid
grid = [
    ['S', '.', '#', '.', 'K'],
    ['.', '#', '.', '#', '.'],
    ['.', '.', '.', '.', '.'],
    ['#', '.', '#', '.', '#'],
    ['.', '.', '.', '.', '.']
]

env = Environment(grid)
agent = GoalBasedAgent(env)
path = agent.bfs()

print("Path to the key:", path)


Path to the key: [(0, 0), (1, 0), (2, 0), (2, 1), (2, 2), (2, 3), (2, 4), (1, 4), (0, 4)]


In [10]:
import heapq

class GBFS_Agent:
    def __init__(self, environment):
        self.environment = environment

    def heuristic(self, pos):
        return abs(pos[0] - self.environment.key[0]) + abs(pos[1] - self.environment.key[1])

    def generate_neighbors(self, pos):
        directions = [(-1, 0), (1, 0), (0, -1), (0, 1)]
        neighbors = []

        for dx, dy in directions:
            nx, ny = pos[0] + dx, pos[1] + dy
            if self.environment.is_valid(nx, ny):
                neighbors.append((nx, ny))

        return neighbors

    def gbfs(self):
        priority_queue = []
        heapq.heappush(priority_queue, (0, [self.environment.start]))
        visited = set()

        while priority_queue:
            _, path = heapq.heappop(priority_queue)
            current = path[-1]

            if current in visited:
                continue

            if current == self.environment.key:
                return path

            visited.add(current)

            for neighbor in self.generate_neighbors(current):
                if neighbor not in visited:
                    new_path = path + [neighbor]
                    heapq.heappush(priority_queue, (self.heuristic(neighbor), new_path))

        return None

# Execute GBFS
gbfs_agent = GBFS_Agent(env)
path = gbfs_agent.gbfs()

print("GBFS Path to the key:", path)


GBFS Path to the key: [(0, 0), (1, 0), (2, 0), (2, 1), (2, 2), (2, 3), (2, 4), (1, 4), (0, 4)]


In [11]:
import random

def initialize_population(size, book_count):
    return [random.sample(range(book_count), book_count) for _ in range(size)]

def fitness(solution):
    conflicts = sum([abs(solution[i] - solution[i + 1]) == 1 for i in range(len(solution) - 1)])
    return 1 / (1 + conflicts)

def roulette_selection(population):
    total_fitness = sum(fitness(ind) for ind in population)
    probabilities = [fitness(ind) / total_fitness for ind in population]
    return population[random.choices(range(len(population)), probabilities)[0]]

def crossover(parent1, parent2):
    point1, point2 = sorted(random.sample(range(len(parent1)), 2))
    child = parent1[:point1] + parent2[point1:point2] + parent1[point2:]
    return child

def mutate(individual):
    index = random.randint(0, len(individual) - 1)
    individual[index] = random.choice(range(len(individual)))

# GA Execution
population_size = 5
books = 5
generations = 10

population = initialize_population(population_size, books)

for _ in range(generations):
    new_population = []
    for _ in range(population_size):
        parent1 = roulette_selection(population)
        parent2 = roulette_selection(population)
        child = crossover(parent1, parent2)
        mutate(child)
        new_population.append(child)

    population = new_population

best_solution = max(population, key=fitness)
print("Best Solution:", best_solution)


Best Solution: [0, 0, 2, 2, 0]
